# Building Interactive Dashboards with Streamlit
Welcome to the Streamlit module! In this notebook, we'll learn how to create interactive web applications for data analysis.

## What you'll learn:
1. Setting up a Streamlit app
2. Creating interactive widgets
3. Displaying data and charts
4. Building a sales dashboard

## Basic Streamlit App
Here's a simple example of a Streamlit app (save this as `sales_dashboard.py`):

In [3]:
import streamlit as st
import pandas as pd
import plotly.express as px

# Page config
st.set_page_config(page_title='Sales Dashboard', layout='wide')

# Title
st.title('Sales Performance Dashboard')

# Load data
@st.cache_data
def load_data():
    return pd.read_csv('../2_pandas_fundamentals/sales_analysis.csv')

df = load_data()

# Sidebar filters
st.sidebar.header('Filters')
selected_products = st.sidebar.multiselect(
    'Select Products',
    options=df['Product'].unique(),
    default=df['Product'].unique()
)

# Filter data
filtered_df = df[df['Product'].isin(selected_products)]

# Create columns for metrics
col1, col2, col3 = st.columns(3)

with col1:
    total_revenue = filtered_df['Total Revenue'].sum()
    st.metric('Total Revenue', f'${total_revenue:,.0f}')

with col2:
    total_units = filtered_df['Total Units'].sum()
    st.metric('Total Units Sold', f'{total_units:,}')

with col3:
    avg_price = total_revenue / total_units
    st.metric('Average Price', f'${avg_price:.2f}')

# Create charts
col1, col2 = st.columns(2)

with col1:
    # Revenue by product
    fig1 = px.bar(filtered_df, x='Product', y='Total Revenue',
                 title='Revenue by Product')
    st.plotly_chart(fig1)

with col2:
    # Monthly sales trends
    monthly_data = pd.melt(filtered_df, 
                          id_vars=['Product'],
                          value_vars=['January', 'February', 'March'],
                          var_name='Month', value_name='Units')
    fig2 = px.line(monthly_data, x='Month', y='Units', color='Product',
                  title='Monthly Sales Trends')
    st.plotly_chart(fig2)

# Show raw data
if st.checkbox('Show Raw Data'):
    st.write(filtered_df)

2025-02-04 19:18:21.115 No runtime found, using MemoryCacheStorageManager


To run this app:
1. Save the code as `sales_dashboard.py`
2. Install required packages: `pip install streamlit plotly`
3. Run: `streamlit run sales_dashboard.py`

## Exercise
Create a Streamlit dashboard that:
1. Allows users to upload their own CSV file
2. Provides basic statistical analysis
3. Generates customizable visualizations
4. Includes data filtering options

In [ ]:
# Example solution (save as data_analyzer.py):
import streamlit as st
import pandas as pd
import plotly.express as px
import seaborn as sns

st.title('Interactive Data Analyzer')

# File upload
uploaded_file = st.file_uploader('Choose a CSV file', type='csv')

if uploaded_file is not None:
    # Load data
    df = pd.read_csv(uploaded_file)
    
    # Sidebar controls
    st.sidebar.header('Analysis Options')
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    
    # Select columns for analysis
    selected_cols = st.sidebar.multiselect(
        'Select columns for analysis',
        options=numeric_cols,
        default=numeric_cols[0] if len(numeric_cols) > 0 else None
    )
    
    if selected_cols:
        # Basic statistics
        st.header('Statistical Summary')
        st.write(df[selected_cols].describe())
        
        # Visualization options
        st.header('Visualization')
        plot_type = st.selectbox(
            'Select plot type',
            ['Histogram', 'Box Plot', 'Scatter Plot', 'Correlation Heatmap']
        )
        
        if plot_type == 'Histogram':
            col = st.selectbox('Select column', selected_cols)
            fig = px.histogram(df, x=col)
            st.plotly_chart(fig)
            
        elif plot_type == 'Box Plot':
            col = st.selectbox('Select column', selected_cols)
            fig = px.box(df, y=col)
            st.plotly_chart(fig)
            
        elif plot_type == 'Scatter Plot':
            x_col = st.selectbox('Select X axis', selected_cols)
            y_col = st.selectbox('Select Y axis', selected_cols)
            fig = px.scatter(df, x=x_col, y=y_col)
            st.plotly_chart(fig)
            
        elif plot_type == 'Correlation Heatmap':
            fig = px.imshow(df[selected_cols].corr(),
                          text_auto=True,
                          aspect='auto')
            st.plotly_chart(fig)
        
        # Show filtered data
        st.header('Filtered Data')
        st.write(df[selected_cols])